<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/5_flower_dataset_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
sweep_config = {

    'method' : 'grid',
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'
                },
    'parameters' : {
        'batch_size': {'values': [8,16]},
        'learning_rate': {'values': [0.001,0.0001]},
        'hidden_nodes': {'values': [128,64]},
        'img_size': {'values': [16, 224]},
        'epochs': {'values': [5,10]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-new-v2")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: chlcahoq
Sweep URL: https://wandb.ai/praveenkumar-aiml27-jntuh-university/5-flowers-new-v2/sweeps/chlcahoq


In [3]:
def train():
  with wandb.init() as run:
    config = wandb.config

    # Constants
    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
        img_bytes = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, resize_dims)
        return img

    def parse_csvline(csv_line):
        record_default = ["", ""]
        filename, label_string = tf.io.decode_csv(csv_line, record_default)

        img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

        # Convert label string to an integer index
        label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]

        return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )


    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        keras.layers.Dense(config.hidden_nodes, activation="relu"),
        keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    ])

    model.compile(
          optimizer="adam",
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=["accuracy"]
      )

    model.fit(
      train_dataset,
      validation_data=eval_dataset,
      epochs=config.epochs,
      callbacks=[WandbMetricsLogger(log_freq=5)]
      )

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: tt7bw3gc with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: praveenkumar-aiml27 (praveenkumar-aiml27-jntuh-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
    413/Unknown 737s 2s/step - accuracy: 0.3376 - loss: 1.5188

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 818s 2s/step - accuracy: 0.3377 - loss: 1.5185 - val_accuracy: 0.4649 - val_loss: 1.2367
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 728s 2s/step - accuracy: 0.4373 - loss: 1.2622 - val_accuracy: 0.4703 - val_loss: 1.2219
Epoch 3/5
